In [1]:
import twodlearn as tdl
import twodlearn.debug
import twodlearn.datasets.mnist
import twodlearn.datasets.cifar10
import tensorflow as tf
from functools import reduce

In [2]:
dataset = tdl.datasets.mnist.MnistDataset(work_directory='data/mnist', reshape=False)
#dataset = tdl.datasets.cifar10.Cifar10(work_directory='data/cifar10')
n_classes = dataset.train.y.shape[-1]
input_shape = dataset.train.x.shape[1:]

File already downloaded.
File already downloaded.
File already downloaded.
File already downloaded.
loading data/mnist/train-images-idx3-ubyte.gz
loading data/mnist/t10k-images-idx3-ubyte.gz


In [3]:
stacked = tdl.stacked.StackedLayers(name='classifier')
with tf.name_scope(stacked.scope):
    stacked.add(tdl.convnet.Conv2DLayer(kernel_size=[5,5], filters=32,
                                        bias={'trainable': True},
                                        input_shape=(None,)+input_shape))
    stacked.add(tf.keras.layers.ReLU())
    stacked.add(tf.keras.layers.MaxPool2D(pool_size=[2,2]))
    stacked.add(tdl.convnet.Conv2DLayer(kernel_size=[5,5], filters=64))
    stacked.add(tf.keras.layers.MaxPool2D(pool_size=[2,2]))
    stacked.add(tf.keras.layers.ReLU())
    conv_dims = stacked.compute_output_shape(input_shape=(None,)+input_shape)
    reduce_dims = reduce(lambda x, y: x*y, conv_dims.as_list()[1:])
    stacked.add(tf.keras.layers.Reshape((reduce_dims,)))
    stacked.add(tdl.feedforward.AffineLayer(units=512))
    #stacked.add(tf.keras.layers.Dense(units=512))
    stacked.add(tf.keras.layers.ReLU())
    stacked.add(tf.keras.layers.Dense(units=n_classes))
    stacked.add(tf.keras.layers.Softmax())
    
#@tdl.debug.stop_at_error
#def test():
inputs = tf.keras.Input(shape=input_shape)
predictions = stacked(inputs)
model = tf.keras.Model(inputs=inputs, outputs=predictions)
#test()

In [4]:
model.compile(optimizer=tf.train.AdamOptimizer(0.002),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
tdl.core.variables_initializer(tdl.core.get_variables(stacked))\
   .run(session=tf.keras.backend.get_session())

logger = tf.keras.callbacks.TensorBoard()
logger.set_model(model)


In [5]:
model.fit(dataset.train.x, dataset.train.y, batch_size=32, epochs=2)

Epoch 1/2
54000/54000 [==============================] - 11s 212us/step - loss: 0.1076 - acc: 0.9673
Epoch 2/2
54000/54000 [==============================] - 10s 192us/step - loss: 0.0427 - acc: 0.9870


In [6]:
_, train_accuracy = model.evaluate(dataset.train.x, dataset.train.y)
_, test_accuracy = model.evaluate(dataset.valid.x, dataset.valid.y)

6000/6000 [==============================] - 1s 95us/step


In [12]:
assert (train_accuracy>0.98) and ((test_accuracy>0.97)), "Accurary of convnet test lower than expected"